In [64]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [65]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margin = df_margin.loc[df_margin['day'] == df_margin['day'].max()].copy()

200


In [66]:
query = """
WITH orders AS (
SELECT DISTINCT
    DATE(fs.order_submitted_date) AS submit_date,
    dp.card_id,
    MAX(fs.order_id) AS order_id
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
WHERE 
    DATE(fs.order_submitted_date) BETWEEN current_date - 20 AND current_date
    AND fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
GROUP BY 1,2
)

SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    dp.card_id,
    dp.card_description AS product_name,
    fs.product_price AS product_price,
    product_price_discount AS product_price_discount,
    pb.price,
    fs.product_price*100.00/pb.price AS pi,
    product_price_discount*100.00/pb.price AS npi
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     
LEFT JOIN dpr_shared.dim_stock_unit         su  ON su.card_id = dp.card_id
LEFT JOIN dpr_pricing.snap_daily_benchmark_competitor pb ON pb.dim_stock_unit = su.stock_unit_id AND fs.dim_submitted_date = pb.dim_benchmark_day
WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND pb.price IS NOT NULL
    AND (fs.order_id,dp.card_id) IN (SELECT DISTINCT order_id, card_id FROM orders)
"""

df_price = run_read_dwd_query(query)
df_price[["product_price","product_price_discount"]] = df_price[["product_price","product_price_discount"]].astype(float)
df_price['submit_date'] = pd.to_datetime(df_price['submit_date'])

In [67]:
query = """
SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    dp.card_id,
    --dp.card_description AS product_name,
    SUM(fs.product_quantity_x_step_unit) AS cant,
    SUM(fs.gmv_pxq_local) AS gmv_usd,
    COALESCE(SUM(fsd.product_discount),0) AS discount_applied
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     
LEFT JOIN dpr_shared.dim_stock_unit         su  ON su.card_id = dp.card_id
LEFT JOIN dpr_pricing.snap_daily_benchmark_competitor pb ON pb.dim_stock_unit = su.stock_unit_id AND fs.dim_submitted_date = pb.dim_benchmark_day
WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND pb.price IS NOT NULL
    AND DATE(fs.order_submitted_date) BETWEEN current_date - 20 AND current_date
    AND fsd.adjustment_reason ILIKE 'hook%'
GROUP BY 1,2,3--,4
"""
df_sells = run_read_dwd_query(query)
df_sells[["cant","gmv_usd","discount_applied"]] = df_sells[["cant","gmv_usd","discount_applied"]].astype(float)
df_sells['submit_date'] = pd.to_datetime(df_sells['submit_date'])

In [68]:
df = pd.merge(df_price, df_sells,  how='inner', left_on=['region_code','submit_date','card_id'], right_on = ['region_code','submit_date','card_id'])
df = pd.merge(df, df_margin[['region','id_tarjeta','margin']],  how='inner', left_on=['region_code','card_id'], right_on = ['region','id_tarjeta'])

df = df.drop(["region", "id_tarjeta"], axis=1)
df = df.sort_values(["card_id", "submit_date"])

In [69]:
df['hook'] = round(1-(df.product_price_discount/df.product_price),4)
df["cash_margin"] = df.gmv_usd * df.margin
df["net_cash_margin"] = df.cash_margin - df.discount_applied
df["roi"] = (df.cash_margin - df.discount_applied) / df.discount_applied

In [70]:
df.columns

Index(['region_code', 'submit_date', 'card_id', 'product_name',
       'product_price', 'product_price_discount', 'price', 'pi', 'npi', 'cant',
       'gmv_usd', 'discount_applied', 'margin', 'hook', 'cash_margin',
       'net_cash_margin', 'roi'],
      dtype='object')

In [71]:
df.loc[df.card_id == -200916]

,region_code,submit_date,card_id,product_name,product_price,product_price_discount,price,pi,npi,cant,gmv_usd,discount_applied,margin,hook,cash_margin,net_cash_margin,roi
6987,BOG,2023-06-22,-200916,Azúcar Bulto x 50kg,206686.93,194285.71,202000.0,102.320262,96.181045,588.0,1.216984e+08,7297774.30,0.047071,0.0600,5.728466e+06,-1.569308e+06,-0.215039
6978,BOG,2023-06-23,-200916,Azúcar Bulto x 50kg,206686.93,194285.71,199000.0,103.862779,97.631010,434.0,8.970827e+07,4795491.37,0.047071,0.0600,4.222657e+06,-5.728342e+05,-0.119453
6975,BOG,2023-06-24,-200916,Azúcar Bulto x 50kg,206686.93,198419.45,199000.0,103.862779,99.708266,248.0,5.124453e+07,2072436.70,0.047071,0.0400,2.412131e+06,3.396941e+05,0.163910
6980,BOG,2023-06-25,-200916,Azúcar Bulto x 50kg,206686.93,198419.45,199000.0,103.862779,99.708266,113.0,2.335562e+07,938358.98,0.047071,0.0400,1.099372e+06,1.610134e+05,0.171590
6986,BOG,2023-06-26,-200916,Azúcar Bulto x 50kg,203460.48,194304.76,199000.0,102.241447,97.640583,566.0,1.159709e+08,5007199.79,0.047071,0.0450,5.458866e+06,4.516663e+05,0.090203
6972,BOG,2023-06-27,-200916,Azúcar Bulto x 50kg,203460.48,194304.76,197000.0,103.279431,98.631858,484.0,9.848455e+07,4435450.20,0.047071,0.0450,4.635766e+06,2.003154e+05,0.045162
6984,BOG,2023-06-28,-200916,Azúcar Bulto x 50kg,199451.51,190476.19,197000.0,101.244421,96.688421,675.0,1.352444e+08,6214511.40,0.047071,0.0450,6.366089e+06,1.515776e+05,0.024391
6988,BOG,2023-06-29,-200916,Azúcar Bulto x 50kg,198904.96,187965.19,197000.0,100.966985,95.413802,679.0,1.352127e+08,7031060.16,0.047071,0.0550,6.364595e+06,-6.664655e+05,-0.094789
6974,BOG,2023-06-30,-200916,Azúcar Bulto x 50kg,196895.72,186657.14,195000.0,100.972164,95.721610,645.0,1.273866e+08,6621318.58,0.047071,0.0520,5.996212e+06,-6.251062e+05,-0.094408
6983,BOG,2023-07-01,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,195000.0,100.972164,96.529390,540.0,1.063399e+08,4680529.98,0.047071,0.0440,5.005525e+06,3.249947e+05,0.069435


In [62]:
import pandas as pd

# Assuming you have a DataFrame 'df2' containing the product information for the last 15 days
# Make sure the DataFrame is sorted by 'submit_date' in ascending order

# Define the initial discount percentage and the increment value
initial_discount = 10
discount_increment = 1

# Iterate over the DataFrame to calculate ROI for different discount levels
for i in range(len(df2)):
    # Calculate the ROI for the current discount level
    discount_percentage = initial_discount + (i * discount_increment)
    df2['discount_applied'] = df2['product_price'] * (discount_percentage / 100)
    df2['margin'] = df2['gmv_usd'] - df2['discount_applied']
    df2['roi'] = (df2['margin'] / df2['gmv_usd']) * 100
    
    # Determine the optimal discount for the current day based on the previous 15 days' data
    previous_data = df2[max(0, i-15):i]
    optimal_discount = previous_data['discount_applied'].mean()  # Adjust this based on your specific optimization criteria
    
    # Update the discount for the current day
    df2.at[i, 'discount_applied'] = optimal_discount
    
    # Calculate the ROI with the optimal discount
    df2.at[i, 'margin'] = df2.at[i, 'gmv_usd'] - optimal_discount
    df2.at[i, 'roi'] = (df2.at[i, 'margin'] / df2.at[i, 'gmv_usd']) * 100

# Print the DataFrame with the suggested discounts and calculated ROIs
print(df2[['submit_date', 'product_name', 'discount_applied', 'roi']])

  submit_date         product_name  discount_applied        roi
0  2023-07-02  Azúcar Bulto x 50kg        37410.1868  99.909953
1  2023-07-03  Azúcar Bulto x 50kg        37410.1868  99.948371
2  2023-07-04  Azúcar Bulto x 50kg        37410.1868  99.971734
3  2023-07-05  Azúcar Bulto x 50kg        37095.2390  99.971643
4  2023-07-06  Azúcar Bulto x 50kg        37095.2390  99.966132
5  2023-07-07  Azúcar Bulto x 50kg        37095.2390  99.959402
6  2023-07-08  Azúcar Bulto x 50kg        37095.2390  99.941536
7  2023-07-09  Azúcar Bulto x 50kg        37095.2390  99.861314
8  2023-07-10  Azúcar Bulto x 50kg        37095.2390  99.966192
9  2023-07-11  Azúcar Bulto x 50kg        37200.2216  99.959596


In [63]:
df2

,region_code,submit_date,card_id,product_name,product_price,product_price_discount,price,pi,npi,cant,gmv_usd,discount_applied,margin,hook,cash_margin,net_cash_margin,roi
0,BOG,2023-07-02,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,195000.0,100.972164,96.529390,211.0,4.154500e+07,37410.1868,4.150759e+07,0.0440,1.955564e+06,127584.734830,99.909953
1,BOG,2023-07-03,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,195000.0,100.972164,96.529390,368.0,7.245963e+07,37410.1868,7.242222e+07,0.0440,3.410747e+06,222523.619137,99.948371
2,BOG,2023-07-04,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,194000.0,101.492639,97.026964,675.0,1.323510e+08,37410.1868,1.323136e+08,0.0440,6.229891e+06,406448.214565,99.971734
3,BOG,2023-07-05,-200916,Azúcar Bulto x 50kg,195238.10,186647.62,194000.0,100.638196,96.210113,670.0,1.308145e+08,37095.2390,1.307774e+08,0.0440,6.157568e+06,400947.051942,99.971643
4,BOG,2023-07-06,-200916,Azúcar Bulto x 50kg,195238.10,186647.62,194000.0,100.638196,96.210113,561.0,1.095302e+08,37095.2390,1.094931e+08,0.0440,5.155697e+06,261784.992679,99.966132
5,BOG,2023-07-07,-200916,Azúcar Bulto x 50kg,195238.10,186452.39,194000.0,100.638196,96.109479,468.0,9.137143e+07,37095.2390,9.133434e+07,0.0450,4.300944e+06,190403.047969,99.959402
6,BOG,2023-07-08,-200916,Azúcar Bulto x 50kg,195238.10,184929.53,194000.0,100.638196,95.324500,325.0,6.344976e+07,37095.2390,6.341266e+07,0.0528,2.986643e+06,131405.428593,99.941536
7,BOG,2023-07-09,-200916,Azúcar Bulto x 50kg,195238.10,186452.39,194000.0,100.638196,96.109479,137.0,2.674762e+07,37095.2390,2.671052e+07,0.0450,1.259037e+06,55394.740410,99.861314
8,BOG,2023-07-10,-200916,Azúcar Bulto x 50kg,195238.10,186452.39,194000.0,100.638196,96.109479,562.0,1.097238e+08,37095.2390,1.096867e+08,0.0450,5.164809e+06,227239.738031,99.966192
9,BOG,2023-07-11,-200916,Azúcar Bulto x 50kg,193925.70,185199.04,193500.0,100.220000,95.710098,474.0,9.207167e+07,37200.2216,9.203447e+07,0.0450,4.333905e+06,190680.114844,99.959596


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

def predict_hook(df):
  """
  Predicts the best value of Hook that maximizes the ROI and net_cash_margin.

  Args:
    df: A pandas DataFrame containing the product data.

  Returns:
    The predicted value of Hook.
  """

  # Split the data into training and test sets.
  X_train, X_test, y_train, y_test = train_test_split(df[["pi", "npi", "roi", "net_cash_margin"]], df["hook"], test_size=0.25)

  # Create a linear regression model.
  model = LinearRegression()
  model.fit(X_train, y_train)

  # Predict the hook values for the test set.
  predicted_hooks = model.predict(X_test)

  # Calculate the mean ROI and net_cash_margin for the predicted hooks.
  mean_roi = np.mean(y_test[predicted_hooks == np.max(predicted_hooks)])
  mean_net_cash_margin = np.mean(y_test[predicted_hooks == np.max(predicted_hooks)]["net_cash_margin"])

  # Return the predicted value of Hook.
  return np.max(predicted_hooks)

In [58]:
import pandas as pd
from pulp import *

# Read the DataFrame
df2 = df.loc[df.card_id == -200916].reset_index(drop=True).copy()  # replace with the actual path to your CSV file

# Create the optimization problem
prob = LpProblem("Discount Optimization", LpMaximize)

# Define the decision variables
discount_vars = LpVariable.dicts("Discount", df2.index, lowBound=0, upBound=1, cat="Continuous")

# Define the objective function
prob += lpSum((df2["cash_margin"][i] - discount_vars[i] * df2["gmv_usd"][i]) for i in df2.index)

# Define the constraints
prob += lpSum(df2["margin"][i] - discount_vars[i] for i in df2.index) >= 0  # Ensure positive margin
prob += lpSum(discount_vars[i] for i in df2.index) <= lpSum(df2["margin"][i] for i in df2.index)  # Ensure margin is greater than or equal to the discount applied
prob += lpSum(df2["pi"][i] - discount_vars[i] for i in df2.index) >= 0.95 * lpSum(discount_vars[i] for i in df2.index)  # Ensure net price index is at least 95% of total discount applied
prob += lpSum(df2["pi"][i] - discount_vars[i] for i in df2.index) <= 1.15 * lpSum(discount_vars[i] for i in df2.index)  # Ensure net price index is at most 115% of total discount applied

# Set the constraint for the sum of optimized discounts
prob += lpSum(discount_vars[i] for i in df2.index) <= 1

# Solve the optimization problem
prob.solve()

# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the optimized discount values
for i in df2.index:
    if discount_vars[i].varValue > 0:
        print("Product ID:", df2["card_id"][i], "Discount:", df2["hook"][i], "Optimized Discount:", discount_vars[i].varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d74dd28c797346978568e5ab7f6ffc40-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/d74dd28c797346978568e5ab7f6ffc40-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 71 RHS
At line 77 BOUNDS
At line 88 ENDATA
Problem MODEL has 5 rows, 10 columns and 50 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 468.59821 (1)
Primal infeasible - objective value -1.9453114e+10
PrimalInfeasible objective -1.945311364e+10 - 1 iterations time 0.002

Result - Linear relaxation infeasible

Enumerated nodes:           0
Total iterations:           0
Time (CPU 

In [35]:
discount_vars[6].varValue

0.0

In [18]:
df2

,region_code,submit_date,card_id,product_name,product_price,product_price_discount,price,pi,npi,cant,gmv_usd,discount_applied,margin,hook,cash_margin,net_cash_margin,roi
0,BOG,2023-07-01,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,195000.0,100.972164,96.529390,540.0,1.063399e+08,4680529.98,0.047713,4.40,5.073846e+06,393316.395044,8.403245
1,BOG,2023-07-02,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,195000.0,100.972164,96.529390,211.0,4.154500e+07,1827979.51,0.047713,4.40,1.982256e+06,154276.719029,8.439740
2,BOG,2023-07-03,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,195000.0,100.972164,96.529390,368.0,7.245963e+07,3188223.29,0.047713,4.40,3.457301e+06,269077.754520,8.439740
3,BOG,2023-07-04,-200916,Azúcar Bulto x 50kg,196895.72,188232.31,194000.0,101.492639,97.026964,675.0,1.323510e+08,5823443.13,0.047713,4.40,6.314925e+06,491481.555590,8.439707
4,BOG,2023-07-05,-200916,Azúcar Bulto x 50kg,195238.10,186647.62,194000.0,100.638196,96.210113,670.0,1.308145e+08,5756621.31,0.047713,4.40,6.241615e+06,484993.238536,8.424963
5,BOG,2023-07-06,-200916,Azúcar Bulto x 50kg,195238.10,186647.62,194000.0,100.638196,96.210113,561.0,1.095302e+08,4893911.74,0.047713,4.40,5.226068e+06,332156.383486,6.787135
6,BOG,2023-07-07,-200916,Azúcar Bulto x 50kg,195238.10,186452.39,194000.0,100.638196,96.109479,468.0,9.137143e+07,4110540.90,0.047713,4.50,4.359649e+06,249107.704797,6.060217
7,BOG,2023-07-08,-200916,Azúcar Bulto x 50kg,193925.70,185199.04,194000.0,99.961701,95.463423,325.0,6.344976e+07,2855237.65,0.047713,4.50,3.027409e+06,172170.864998,6.030001
8,BOG,2023-07-09,-200916,Azúcar Bulto x 50kg,195238.10,186452.39,194000.0,100.638196,96.109479,137.0,2.674762e+07,1203642.27,0.047713,4.50,1.276222e+06,72579.650635,6.030002
9,BOG,2023-07-10,-200916,Azúcar Bulto x 50kg,195238.10,184929.53,194000.0,100.638196,95.324500,563.0,1.099191e+08,4946354.73,0.047713,5.28,5.244620e+06,298265.279617,6.030002
